# Hyderabad

In [2]:
import requests, re, numpy as np, pandas as pd
from bs4 import BeautifulSoup

In [3]:
titles = []
ratings = []
no_ratings = []
rat = []
actual_prices = [] 
slashed_prices = []
discount = [] 
taxes = [] 
address = []
links = []
savings_price = []
total_price = [] 

for i in range(1, 5):
    url = f'https://www.oyorooms.com/hotels-in-hyderabad/?page={i}'
    request_header = {'Content-Type': 'text/html; charset=UTF-8','User-Agent': 'Chrome/101.0.0.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/119.0','Accept-Encoding': 'gzip, deflate, br'}
    page = requests.get(url,headers =request_header)
    soup = BeautifulSoup(page.text)

    #titles and Ratings
    for i in soup.find_all("div", class_ = 'oyo-cell--12-col oyo-cell--8-col-tablet oyo-cell--4-col-phone'):
        title = i.find("h3", class_ = "listingHotelDescription__hotelName d-textEllipsis")
        titles.append(title.text)
        
        text = i.find("div", class_ = 'hotelRating__wrapper').text
        if text:
            
            r1 = re.findall('(\d\.\d)',text)
            if r1:
                ratings.append(r1[0])
            else:
                ratings.append(np.nan)

            r2 = re.findall('\((\d+)',text)
            if r2:
                no_ratings.append(r2[0])
            else:
                no_ratings.append(np.nan)

            r3 = re.findall('\·(\w.+)',text)
            if r3:
                rat.append(r3[0])
            else:
                rat.append(np.nan) 

    #prices 
    for i in soup.find_all("div", class_ = 'listingPrice__numbers'):
        ac_price = i.find('span', class_ = 'listingPrice__finalPrice listingPrice__finalPrice--black').text
        sl_price = i.find('span', class_ = 'listingPrice__slashedPrice d-body-lg').text 
        dis = i.find('span', class_ = 'listingPrice__percentage').text
        if ac_price:
            r1 = re.findall('₹(.+)', ac_price)
            if r1:
                actual_prices.append(r1[0])
            else:
                actual_prices.append(np.nan)
        if sl_price:
            r2 = re.findall('₹(.+)',sl_price)
            if r2:
                slashed_prices.append(r2[0])
            else:
                slashed_prices.append(np.nan)
        if dis:
            r3 = re.findall('(.+)%', dis)
            if r3:
                discount.append(r3[0])
            else:
                discount.append(np.nan)

        #taxes        
        for i in soup.find_all("div", class_ = 'listingPrice__perRoomNight'):
            tax = i.text
        if tax:
            r1 = re.findall('₹(\d+)\s?',tax)
            taxes.append(r1[0])
        else:
            taxes.append(np.nan)
            
    #address
    for i in soup.find_all("span", class_ = 'u-line--clamp-2'):
        address.append(i.text) 
    
    #links
    for i in soup.find_all('a',class_="c-nn640c u-width100"):
        link = 'https://www.oyorooms.com' + i.get('href')
        links.append(link)


#prices
for i in links:
    url = i
    request_header = {'Content-Type': 'text/html; charset=UTF-8','User-Agent': 'Chrome/101.0.0.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/119.0','Accept-Encoding': 'gzip, deflate, br'}
    page = requests.get(url,headers =request_header)
    soup = BeautifulSoup(page.text)
    
    for i in soup.find_all('div',class_= 'c-t958pf'):
        text = i.text
        if text:
            r1 = re.findall('Your savings\s?₹(\d+)', text)
            if r1:
                savings_price.append(r1[0]) 
            else:
                savings_price.append(np.nan)

            r2 = re.findall('Total price\s?\s?₹(\d+)', text)
            if r2:
                total_price.append(r2[0]) 
            else:
                total_price.append(np.nan)

In [7]:
dfhyd = pd.DataFrame({'Title':titles,
'Ratings' :ratings,
'NoofRatings' : no_ratings,
'Overall_Rating': rat,
'Actual_prices':actual_prices,
'Slashed_prices': slashed_prices,
'Discount': discount,
"Tax": taxes,
'Savings_price': savings_price,
'Total_price': total_price,
'Address': address,
'City': ['Hyderabad'] * len(titles)})
dfhyd

,Title,Ratings,NoofRatings,Overall_Rating,Actual_prices,Slashed_prices,Discount,Tax,Savings_price,Total_price,Address,City
0,Super Townhouse Oak N7 Elite,4.3,2111,Very Good,1262,4539,68,132,1764,1414,"Champapet, Hyderabad",Hyderabad
1,Super OYO Flagship Vsm Grand,4.6,598,Excellent,864,3237,68,132,1258,1008,"Aprana Sarovar , Gopanpally",Hyderabad
2,Townhouse Secunderabad Near Paradise Metro Sta...,4.5,88,Excellent,1273,5235,72,132,2199,1466,"Jogani, Hyderabad",Hyderabad
3,Super OYO Flagship Boduppal,4.5,2135,Excellent,721,3688,77,132,1257,838,"Near Andhra Bank, Medipally, Hyderabad",Hyderabad
4,OYO Hotel Golden Pride,4.4,1008,Very Good,709,3014,72,132,1266,844,"Near Madhura Nagar, Shamshabad, Hyderabad",Hyderabad
...,...,...,...,...,...,...,...,...,...,...,...,...
75,Flagship Gravity Spaces Gs,3.6,322,Good,732,3069,71,182,1289,860,Puppalaguda Manikonda Municipality LIC Colony ...,Hyderabad
76,Super OYO 40869 White Ridge,4.1,387,Very Good,1033,4239,72,182,1781,1187,"Near Sanjeevini Hospital, Kothapet road, Hyder...",Hyderabad
77,Super Townhouse 269 RCC Sunrise,4.8,1022,Excellent,1262,6454,77,182,1800,1443,"Near Begumpet lifestyle, Kundanbagh Colony, Hy...",Hyderabad
78,Super Capital O Hotel Saharsh Grand Near Shilp...,4.4,905,Very Good,1332,4917,68,182,1910,1532,"Near VVC Mahindra, Kothaguda, Hyderabad",Hyderabad


# Delhi

In [8]:
titles = []
ratings = []
no_ratings = []
rat = []
actual_prices = [] 
slashed_prices = []
discount = [] 
taxes = []
address = []
links = []
savings_price = []
total_price = [] 

for i in range(1, 5):
    url = f'https://www.oyorooms.com/hotels-in-delhi/?page={i}'
    request_header = {'Content-Type': 'text/html; charset=UTF-8','User-Agent': 'Chrome/101.0.0.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/119.0','Accept-Encoding': 'gzip, deflate, br'}
    page = requests.get(url,headers =request_header)
    soup = BeautifulSoup(page.text)

    #titles
    for i in soup.find_all("div", class_ = 'oyo-cell--12-col oyo-cell--8-col-tablet oyo-cell--4-col-phone'):
        title = i.find("h3", class_ = "listingHotelDescription__hotelName d-textEllipsis")
        titles.append(title.text)
        rating_wrapper = i.find("div", class_ = 'hotelRating__wrapper')
        
        if rating_wrapper: 
            text = rating_wrapper.text.strip()
            r1 = re.findall('(\d\.\d)',text)
            if r1:
                ratings.append(r1[0])
            else:
                ratings.append(np.nan)

            r2 = re.findall('\((\d+)',text)
            if r2:
                no_ratings.append(r2[0])
            else:
                no_ratings.append(np.nan)

            r3 = re.findall('\·(\w.+)',text)
            if r3:
                rat.append(r3[0])
            else:
                rat.append(np.nan) 

    #prices 
    for i in soup.find_all("div", class_ = 'listingPrice__numbers'):
        ac_price = i.find('span', class_ = 'listingPrice__finalPrice listingPrice__finalPrice--black').text
        sl_price = i.find('span', class_ = 'listingPrice__slashedPrice d-body-lg').text 
        dis = i.find('span', class_ = 'listingPrice__percentage').text
        if ac_price:
            r1 = re.findall('₹(.+)', ac_price)
            if r1:
                actual_prices.append(r1[0])
            else:
                actual_prices.append(np.nan)
        if sl_price:
            r2 = re.findall('₹(.+)',sl_price)
            if r2:
                slashed_prices.append(r2[0])
            else:
                slashed_prices.append(np.nan)
        if dis:
            r3 = re.findall('(.+)%', dis)
            if r3:
                discount.append(r3[0])
            else:
                discount.append(np.nan)

        #taxes        
        for i in soup.find_all("div", class_ = 'listingPrice__perRoomNight'):
            tax = i.text
        if tax:
            r1 = re.findall('₹(\d+)\s?',tax)
            taxes.append(r1[0])
        else:
            taxes.append(np.nan)
            
    #address
    for i in soup.find_all("span", class_ = 'u-line--clamp-2'):
        address.append(i.text)

    #links
    for i in soup.find_all('a',class_="c-nn640c u-width100"):
        link = 'https://www.oyorooms.com' + i.get('href')
        links.append(link)


#prices
for i in links:
    url = i
    request_header = {'Content-Type': 'text/html; charset=UTF-8','User-Agent': 'Chrome/101.0.0.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/119.0','Accept-Encoding': 'gzip, deflate, br'}
    page = requests.get(url,headers =request_header)
    soup = BeautifulSoup(page.text)
    
    for i in soup.find_all('div',class_= 'c-t958pf'):
        text = i.text
        if text:
            r1 = re.findall('Your savings\s?₹(\d+)', text)
            if r1:
                savings_price.append(r1[0]) 
            else:
                savings_price.append(np.nan)

            r2 = re.findall('Total price\s?\s?₹(\d+)', text)
            if r2:
                total_price.append(r2[0]) 
            else:
                total_price.append(np.nan)

In [13]:
dfdelhi = pd.DataFrame({'Title':titles,
'Ratings' :ratings,
'NoofRatings' : no_ratings,
'Overall_Rating': rat,
'Actual_prices':actual_prices,
'Slashed_prices': slashed_prices,
'Discount': discount,
"Tax": taxes,
'Savings_price': savings_price,
'Total_price': total_price,
'Address': address,
'City': ['Delhi'] * len(titles)})
dfdelhi

,Title,Ratings,NoofRatings,Overall_Rating,Actual_prices,Slashed_prices,Discount,Tax,Savings_price,Total_price,Address,City
0,Super OYO Chand Palace,3.9,135,Good,1271,4699,68,114,1826,1464,"Paharganj, Delhi",Delhi
1,Super Townhouse 810943 Saket Residency Near Sa...,4.1,1719,Very Good,788,3297,72,114,1385,923,"Near KD Plaza, IGNOU Road, Neb Sarai, Saket",Delhi
2,Super Townhouse Rohini Sector 24,4.1,416,Very Good,1516,6171,72,114,2592,1728,"Near Near Dominos Sector 24, Rohini, Dasghara,...",Delhi
3,Townhouse Rohini Sector 23,4.1,102,Very Good,1100,4507,72,114,1893,1262,"Near Rohini Extension, Budh Vihar, Delhi",Delhi
4,OYO Hotel Crystal INN,4.3,425,Very Good,588,2669,73,114,1025,699,near Dwarka mod metro station near Dwarka mod...,Delhi
...,...,...,...,...,...,...,...,...,...,...,...,...
75,OYO AR Residency,4.0,48,Very Good,809,3379,72,117,1420,946,"Vikaspuri, Delhi",Delhi
76,OYO Mehmaan Residency,3.8,110,Good,515,2202,71,117,925,617,Matiala Road behind Aakash Hospital Dwarka sec...,Delhi
77,OYO Flagship Hotel Delhight,4.4,231,Very Good,746,3125,72,117,1313,875,"Block E, Delhi",Delhi
78,OYO Hotel Landmark Inn,4.6,2039,Excellent,1019,4217,71,117,1771,1181,Near Karol Bagh Metro Station,Delhi


# Bangalore

In [14]:
titles = []
ratings = []
no_ratings = []
rat = [] 
actual_prices = [] 
slashed_prices = []
discount = [] 
taxes = [] 
address = []
links = [] 
savings_price = []
total_price = []

for i in range(1,5):
    url = f'https://www.oyorooms.com/hotels-in-bangalore/?page={i}'
    request_header = {'Content-Type': 'text/html; charset=UTF-8','User-Agent': 'Chrome/101.0.0.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/119.0','Accept-Encoding': 'gzip, deflate, br'}
    page = requests.get(url,headers =request_header)
    soup = BeautifulSoup(page.text)

    #titles
    for i in soup.find_all("div", class_ = 'oyo-cell--12-col oyo-cell--8-col-tablet oyo-cell--4-col-phone'):
        title = i.find("h3", class_ = "listingHotelDescription__hotelName d-textEllipsis")
        titles.append(title.text)
        rating_wrapper = i.find("div", class_ = 'hotelRating__wrapper')

        if rating_wrapper: 
            text = rating_wrapper.text
            r1 = re.findall('(\d\.\d)',text) 
            if r1:
                ratings.append(r1[0])
            else:
                ratings.append(np.nan)

            r2 = re.findall('\((\d+)',text)
            if r2:
                no_ratings.append(r2[0])
            else:
                no_ratings.append(np.nan)

            r3 = re.findall('\·(\w.+)',text)
            if r3:
                rat.append(r3[0])
            else:
                rat.append(np.nan)
        else:
            ratings.append(np.nan)
            no_ratings.append(np.nan)
            rat.append(np.nan)

    #prices 
    for i in soup.find_all("div", class_ = 'listingPrice__numbers'):
        ac_price = i.find('span', class_ = 'listingPrice__finalPrice listingPrice__finalPrice--black').text
        sl_price = i.find('span', class_ = 'listingPrice__slashedPrice d-body-lg').text 
        dis = i.find('span', class_ = 'listingPrice__percentage').text
        if ac_price:
            r1 = re.findall('₹(.+)', ac_price)
            if r1:
                actual_prices.append(r1[0])
            else:
                actual_prices.append(np.nan)
        if sl_price:
            r2 = re.findall('₹(.+)',sl_price)
            if r2:
                slashed_prices.append(r2[0])
            else:
                slashed_prices.append(np.nan) 
        if dis:
            r3 = re.findall('(.+)%', dis)
            if r3:
                discount.append(r3[0])
            else:
                discount.append(np.nan)

        #taxes        
        for i in soup.find_all("div", class_ = 'listingPrice__perRoomNight'):
            tax = i.text
        if tax:
            r1 = re.findall('₹(\d+)\s?',tax)
            taxes.append(r1[0])
        else:
            taxes.append(np.nan)
            
    #address
    for i in soup.find_all("span", class_ = 'u-line--clamp-2'):
        address.append(i.text)

    #links
    for i in soup.find_all('a',class_="c-nn640c u-width100"):
        link = 'https://www.oyorooms.com' + i.get('href')
        links.append(link)


#prices
for i in links:
    url = i
    request_header = {'Content-Type': 'text/html; charset=UTF-8','User-Agent': 'Chrome/101.0.0.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/119.0','Accept-Encoding': 'gzip, deflate, br'}
    page = requests.get(url,headers =request_header)
    soup = BeautifulSoup(page.text)
    
    for i in soup.find_all('div',class_= 'c-t958pf'):
        text = i.text
        if text:
            r1 = re.findall('Your savings\s?₹(\d+)', text)
            if r1:
                savings_price.append(r1[0]) 
            else:
                savings_price.append(np.nan)

            r2 = re.findall('Total price\s?\s?₹(\d+)', text) 
            if r2:
                total_price.append(r2[0]) 
            else:
                total_price.append(np.nan) 

In [17]:
dfbnglr = pd.DataFrame({'Title':titles,
'Ratings' :ratings,
'NoofRatings' : no_ratings,
'Overall_Rating': rat,
'Actual_prices':actual_prices,
'Slashed_prices': slashed_prices,
'Discount': discount,
"Tax": taxes,
'Savings_price': savings_price,
'Total_price': total_price,
'Address': address,
'City': ['Bangalore'] * len(titles)})
dfbnglr

,Title,Ratings,NoofRatings,Overall_Rating,Actual_prices,Slashed_prices,Discount,Tax,Savings_price,Total_price,Address,City
0,Super Townhouse HAL Indiranagar Extension,4.2,2444,Very Good,1571,5652,68,178,2196,1761,"Bangalore, Bangalore",Bangalore
1,Super Collection O Singasandra Hosa Road,4.4,336,Very Good,883,3749,72,178,1543,1029,"Singasandra, Bangalore",Bangalore
2,Super Collection O Bannerghatta Road Near IIM ...,4.1,346,Very Good,1371,5057,68,178,1965,1575,"Near IIM Banagalore, Bangalore",Bangalore
3,Super Townhouse Oak A2 Indiranagar 2nd Stage,4.6,2168,Excellent,1915,6889,68,178,2677,2146,"India, Bangalore",Bangalore
4,Super Capital O Champion Lotus Near Mantri Squ...,4.3,666,Very Good,981,4125,72,178,1421,1139,"Near IIFT, Vijayanagar, Bangalore",Bangalore
...,...,...,...,...,...,...,...,...,...,...,...,...
75,OYO Om Sai Suites,4.0,44,Very Good,852,3619,71,180,1520,1014,"Near Government Hospital, HBCS Layout, Bangalore",Bangalore
76,Super Townhouse 264 - Q Rooms BTM Layout Near ...,4.8,968,Excellent,1812,6642,68,180,2581,2069,"Near BTM Water Tank, Mico Layout, Bangalore",Bangalore
77,SPOT ON 701125 Sri Saptagiri Bording & Lodging,3.5,37,Good,410,3128,84,180,802,499,"Dasarahalli Zone Hegganahalli Main Road, Banga...",Bangalore
78,Townhouse 180 Jayanagar,4.9,2345,Fabulous,1039,4384,72,180,1502,1204,"Near Sagar Hospital, Tilak Nagar, Bangalore",Bangalore


# Mumbai

In [18]:
titles = []
ratings = []
no_ratings = [] 
rat = []
actual_prices = [] 
slashed_prices = []
discount = [] 
taxes = []
address = []
links = []
savings_price = []
total_price = []

for i in range(1, 5):
    url = f'https://www.oyorooms.com/hotels-in-mumbai/?page={i}'
    request_header = {'Content-Type': 'text/html; charset=UTF-8','User-Agent': 'Chrome/101.0.0.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/119.0','Accept-Encoding': 'gzip, deflate, br'}
    page = requests.get(url,headers =request_header)
    soup = BeautifulSoup(page.text)

    #titles
    for i in soup.find_all("div", class_ = 'oyo-cell--12-col oyo-cell--8-col-tablet oyo-cell--4-col-phone'):
        title = i.find("h3", class_ = "listingHotelDescription__hotelName d-textEllipsis")
        titles.append(title.text)
        rating_wrapper = i.find("div", class_ = 'hotelRating__wrapper')

        if rating_wrapper: 
            text = rating_wrapper.text
            r1 = re.findall('(\d\.\d)',text)
            if r1:
                ratings.append(r1[0])
            else:
                ratings.append(np.nan)

            r2 = re.findall('\((\d+)',text)
            if r2:
                no_ratings.append(r2[0])
            else:
                no_ratings.append(np.nan)

            r3 = re.findall('\·(\w.+)',text)
            if r3:
                rat.append(r3[0])
            else:
                rat.append(np.nan) 
        else:
            ratings.append(np.nan)
            no_ratings.append(np.nan)
            rat.append(np.nan)

    #prices 
    for i in soup.find_all("div", class_ = 'listingPrice__numbers'):
        ac_price = i.find('span', class_ = 'listingPrice__finalPrice listingPrice__finalPrice--black').text
        sl_price = i.find('span', class_ = 'listingPrice__slashedPrice d-body-lg').text 
        dis = i.find('span', class_ = 'listingPrice__percentage').text
        if ac_price:
            r1 = re.findall('₹(.+)', ac_price)
            if r1:
                actual_prices.append(r1[0])
            else:
                actual_prices.append(np.nan)
        if sl_price:
            r2 = re.findall('₹(.+)',sl_price)
            if r2:
                slashed_prices.append(r2[0])
            else:
                slashed_prices.append(np.nan)
        if dis:
            r3 = re.findall('(.+)%', dis)
            if r3:
                discount.append(r3[0])
            else:
                discount.append(np.nan)

        #taxes        
        for i in soup.find_all("div", class_ = 'listingPrice__perRoomNight'):
            tax = i.text
        if tax:
            r1 = re.findall('₹(\d+)\s?',tax)
            taxes.append(r1[0])
        else:
            taxes.append(np.nan)
            
    #address
    for i in soup.find_all("span", class_ = 'u-line--clamp-2'):
        address.append(i.text)

    #links
    for i in soup.find_all('a',class_="c-nn640c u-width100"):
        link = 'https://www.oyorooms.com' + i.get('href')
        links.append(link)


#prices
for i in links:
    url = i
    request_header = {'Content-Type': 'text/html; charset=UTF-8','User-Agent': 'Chrome/101.0.0.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/119.0','Accept-Encoding': 'gzip, deflate, br'}
    page = requests.get(url,headers =request_header)
    soup = BeautifulSoup(page.text)
    
    for i in soup.find_all('div',class_= 'c-t958pf'):
        text = i.text
        if text:
            r1 = re.findall('Your savings\s?₹(\d+)', text)
            if r1:
                savings_price.append(r1[0]) 
            else:
                savings_price.append(np.nan)

            r2 = re.findall('Total price\s?\s?₹(\d+)', text)
            if r2:
                total_price.append(r2[0]) 
            else:
                total_price.append(np.nan) 

In [22]:
dfmumbai = pd.DataFrame({'Title':titles,
'Ratings' :ratings,
'NoofRatings' : no_ratings,
'Overall_Rating': rat,
'Actual_prices':actual_prices,
'Slashed_prices': slashed_prices,
'Discount': discount,
"Tax": taxes,
'Savings_price': savings_price,
'Total_price': total_price,
'Address': address,
'City': ['Mumbai'] * len(titles)}) 
dfmumbai

,Title,Ratings,NoofRatings,Overall_Rating,Actual_prices,Slashed_prices,Discount,Tax,Savings_price,Total_price,Address,City
0,Super Collection O Hotel Phoenix,4.2,278,Very Good,986,4368,74,222,1702,1134,"India, Mumbai",Mumbai
1,OYO Hotel Mayur Residency,3.6,20,Good,1132,4165,68,222,1618,1298,"Kalyan-Murbad Road, Village Kamba, Next to Mer...",Mumbai
2,OYO Flagship Yatri Inn,4.2,526,Very Good,1049,5338,77,222,1822,1215,"Sector 26, Mumbai",Mumbai
3,Flagship Hotel Yatri Yadgar,3.8,513,Good,873,3672,72,222,1543,1028,"Vashi Navi Mumbai, Mumbai",Mumbai
4,Super OYO Flagship Hotel Golden Nest,4.5,75,Excellent,992,3964,70,222,1435,1151,"Boisar east, Mumbai",Mumbai
...,...,...,...,...,...,...,...,...,...,...,...,...
75,OYO Druv's Taj Palace Near Chhatrapati Shivaji...,3.9,329,Good,1304,5460,72,277,2251,1501,"Near Asalpha Metro Station, Dhumai Estate, Mumbai",Mumbai
76,OYO Hotel Kishan,4.5,14,Excellent,1355,5565,72,277,2338,1558,"Pant Nagar, Mumbai",Mumbai
77,Flagship Hotel Ashwi,3.8,244,Good,1098,4534,71,277,1904,1270,"MIDC Industrial Area, Mumbai",Mumbai
78,Collection O Collection O Hotel Jayshree Domes...,4.1,258,Very Good,1954,7925,72,277,3329,2219,"Vile Parle East, Mumbai",Mumbai


# chennai

In [23]:
titles = []
ratings = []
no_ratings = []
rat = []
actual_prices = []  
slashed_prices = []
discount = [] 
taxes = []
address = []
links = []
savings_price = []
total_price = []   

for i in range(1, 5):
    url = f'https://www.oyorooms.com/hotels-in-chennai/?page={i}'
    request_header = {'Content-Type': 'text/html; charset=UTF-8','User-Agent': 'Chrome/101.0.0.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/119.0','Accept-Encoding': 'gzip, deflate, br'}
    page = requests.get(url,headers =request_header)
    soup = BeautifulSoup(page.text)

    #titles
    for i in soup.find_all("div", class_ = 'oyo-cell--12-col oyo-cell--8-col-tablet oyo-cell--4-col-phone'):
        title = i.find("h3", class_ = "listingHotelDescription__hotelName d-textEllipsis")
        titles.append(title.text)
        rating_wrapper = i.find("div", class_ = 'hotelRating__wrapper')

        if rating_wrapper: 
            text = rating_wrapper.text
            r1 = re.findall('(\d\.\d)',text)
            if r1:
                ratings.append(r1[0])
            else:
                ratings.append(np.nan)

            r2 = re.findall('\((\d+)',text)
            if r2:
                no_ratings.append(r2[0])
            else:
                no_ratings.append(np.nan)

            r3 = re.findall('\·(\w.+)',text)
            if r3:
                rat.append(r3[0])
            else:
                rat.append(np.nan) 
        else:
            ratings.append(np.nan)
            no_ratings.append(np.nan)
            rat.append(np.nan)

    #prices 
    for i in soup.find_all("div", class_ = 'listingPrice__numbers'):
        ac_price = i.find('span', class_ = 'listingPrice__finalPrice listingPrice__finalPrice--black').text
        sl_price = i.find('span', class_ = 'listingPrice__slashedPrice d-body-lg').text 
        dis = i.find('span', class_ = 'listingPrice__percentage').text
        if ac_price:
            r1 = re.findall('₹(.+)', ac_price)
            if r1:
                actual_prices.append(r1[0])
            else:
                actual_prices.append(np.nan)
        if sl_price:
            r2 = re.findall('₹(.+)',sl_price)
            if r2:
                slashed_prices.append(r2[0])
            else:
                slashed_prices.append(np.nan)
        if dis:
            r3 = re.findall('(.+)%', dis)
            if r3:
                discount.append(r3[0])
            else:
                discount.append(np.nan)

        #taxes        
        for i in soup.find_all("div", class_ = 'listingPrice__perRoomNight'):
            tax = i.text
        if tax:
            r1 = re.findall('₹(\d+)\s?',tax)
            taxes.append(r1[0])
        else:
            taxes.append(np.nan)
            
    #address
    for i in soup.find_all("span", class_ = 'u-line--clamp-2'):
        address.append(i.text)

    #links
    for i in soup.find_all('a',class_="c-nn640c u-width100"):
        link = 'https://www.oyorooms.com' + i.get('href')
        links.append(link)


#prices
for i in links:
    url = i
    request_header = {'Content-Type': 'text/html; charset=UTF-8','User-Agent': 'Chrome/101.0.0.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/119.0','Accept-Encoding': 'gzip, deflate, br'}
    page = requests.get(url,headers =request_header)
    soup = BeautifulSoup(page.text) 
    
    for i in soup.find_all('div',class_= 'c-t958pf'):
        text = i.text
        if text:
            r1 = re.findall('Your savings\s?₹(\d+)', text)
            if r1:
                savings_price.append(r1[0]) 
            else:
                savings_price.append(np.nan)

            r2 = re.findall('Total price\s?\s?₹(\d+)', text)
            if r2:
                total_price.append(r2[0]) 
            else:
                total_price.append(np.nan)

In [26]:
dfchennai = pd.DataFrame({'Title':titles,
'Ratings' :ratings,
'NoofRatings' : no_ratings,
'Overall_Rating': rat,
'Actual_prices':actual_prices,
'Slashed_prices': slashed_prices,
'Discount': discount,
"Tax": taxes, 
'Savings_price': savings_price,
'Total_price': total_price,
'Address': address,
'City': ['Chennai'] * len(titles)})
dfchennai

,Title,Ratings,NoofRatings,Overall_Rating,Actual_prices,Slashed_prices,Discount,Tax,Savings_price,Total_price,Address,City
0,Super Collection O Chepauk Near MA Chidambram ...,4.6,282,Excellent,1052,4242,71,99,1752,1218,"Near Walajah Big Mosque, Triplicane High Road,...",Chennai
1,Flagship Tambaram,4.0,1153,Very Good,715,3002,71,99,1261,841,"Near Tambaram Railwaystation, Tambaram, Chennai",Chennai
2,Collection O Hotel Silver Key Executive,4.5,1528,Excellent,499,2758,78,99,695,599,"Perumbakkam, Chennai",Chennai
3,The Park View Inn,4.0,985,Very Good,444,2993,82,99,805,537,"opp to Indian Oil,Church Road,Vepery.",Chennai
4,Flagship Jayam Residency - Urapakkam,4.2,774,Very Good,568,2412,71,99,1013,676,"Near Vandalur Zoo, Urapakkam, Chennai",Chennai
...,...,...,...,...,...,...,...,...,...,...,...,...
75,Super Townhouse 1060 Anna salai,4.7,1334,Excellent,1105,4071,68,89,1582,1268,"Near US Embassy, Royapettah",Chennai
76,FLAGSHIP Raaj Residency\t\t\t\t\t,3.4,1111,Fair,553,2117,68,89,823,659,"Pallavaram, Chennai",Chennai
77,OYO Collection O G Silver Ashok Nagar,4.6,1503,Excellent,1308,4799,68,89,1865,1495,"600001, Chennai",Chennai
78,Flagship Bella Vagues Kovalam,3.6,62,Good,879,3621,72,89,1521,1014,"Kovalam, Chennai",Chennai


# Gurgaon

In [27]:
titles = []
ratings = []
no_ratings = []
rat = []
actual_prices = [] 
slashed_prices = [] 
discount = [] 
taxes = []
address = []
links = []
savings_price = []
total_price = []

for i in range(1, 5):
    url = f'https://www.oyorooms.com/hotels-in-gurgaon/?page={i}'
    request_header = {'Content-Type': 'text/html; charset=UTF-8','User-Agent': 'Chrome/101.0.0.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/119.0','Accept-Encoding': 'gzip, deflate, br'}
    page = requests.get(url,headers =request_header)
    soup = BeautifulSoup(page.text)

    #titles
    for i in soup.find_all("div", class_ = 'oyo-cell--12-col oyo-cell--8-col-tablet oyo-cell--4-col-phone'):
        title = i.find("h3", class_ = "listingHotelDescription__hotelName d-textEllipsis")
        titles.append(title.text)
        rating_wrapper = i.find("div", class_ = 'hotelRating__wrapper')

        if rating_wrapper: 
            text = rating_wrapper.text
            r1 = re.findall('(\d\.\d)',text)
            if r1:
                ratings.append(r1[0])
            else:
                ratings.append(np.nan)

            r2 = re.findall('\((\d+)',text)
            if r2:
                no_ratings.append(r2[0])
            else:
                no_ratings.append(np.nan)

            r3 = re.findall('\·(\w.+)',text)
            if r3:
                rat.append(r3[0])
            else:
                rat.append(np.nan) 
        else:
            ratings.append(np.nan)
            no_ratings.append(np.nan)
            rat.append(np.nan)

    #prices 
    for i in soup.find_all("div", class_ = 'listingPrice__numbers'):
        ac_price = i.find('span', class_ = 'listingPrice__finalPrice listingPrice__finalPrice--black').text
        sl_price = i.find('span', class_ = 'listingPrice__slashedPrice d-body-lg').text 
        dis = i.find('span', class_ = 'listingPrice__percentage').text
        if ac_price:
            r1 = re.findall('₹(.+)', ac_price)
            if r1:
                actual_prices.append(r1[0])
            else:
                actual_prices.append(np.nan)
        if sl_price:
            r2 = re.findall('₹(.+)',sl_price)
            if r2:
                slashed_prices.append(r2[0])
            else:
                slashed_prices.append(np.nan)
        if dis:
            r3 = re.findall('(.+)%', dis)
            if r3:
                discount.append(r3[0])
            else:
                discount.append(np.nan)

        #taxes        
        for i in soup.find_all("div", class_ = 'listingPrice__perRoomNight'):
            tax = i.text
        if tax:
            r1 = re.findall('₹(\d+)\s?',tax)
            taxes.append(r1[0])
        else:
            taxes.append(np.nan)
            
    #address
    for i in soup.find_all("span", class_ = 'u-line--clamp-2'):
        address.append(i.text)

    #links
    for i in soup.find_all('a',class_="c-nn640c u-width100"):
        link = 'https://www.oyorooms.com' + i.get('href')
        links.append(link)

#prices
for i in links:
    url = i
    request_header = {'Content-Type': 'text/html; charset=UTF-8','User-Agent': 'Chrome/101.0.0.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/119.0','Accept-Encoding': 'gzip, deflate, br'}
    page = requests.get(url,headers =request_header)
    soup = BeautifulSoup(page.text)
    
    for i in soup.find_all('div',class_= 'c-t958pf'):
        text = i.text
        if text:
            r1 = re.findall('Your savings\s?₹(\d+)', text)
            if r1:
                savings_price.append(r1[0]) 
            else:
                savings_price.append(np.nan)

            r2 = re.findall('Total price\s?\s?₹(\d+)', text)
            if r2:
                total_price.append(r2[0]) 
            else:
                total_price.append(np.nan)

In [30]:
dfgurgaon = pd.DataFrame({'Title':titles,
'Ratings' :ratings,
'NoofRatings' : no_ratings,
'Overall_Rating': rat,
'Actual_prices':actual_prices,
'Slashed_prices': slashed_prices,
'Discount': discount,
"Tax": taxes,
'Savings_price': savings_price,
'Total_price': total_price, 
'Address': address,
'City': ['Gurgaon'] * len(titles)})
dfgurgaon

,Title,Ratings,NoofRatings,Overall_Rating,Actual_prices,Slashed_prices,Discount,Tax,Savings_price,Total_price,Address,City
0,Super OYO Hotel Samrat,4.2,1977,Very Good,596,2527,71,195,1061,708,"Near Bus Stand, Sadar Bazar, Gurgaon",Gurgaon
1,Townhouse 1291 Akshay Hotel,4.2,90,Very Good,588,2886,75,195,999,699,"Sec-39, Gurgaon",Gurgaon
2,Garden View Hotel Near Hero Honda Chowk,4.3,1842,Very Good,419,2181,77,195,746,499,"Natcom Institute, Gurgaon",Gurgaon
3,Super Collection O 1064 Tipsyy Inn Suites 17,4.3,227,Very Good,499,2426,75,195,824,599,"Near GD Goenka University, Dhunela, Gurgaon",Gurgaon
4,Super OYO The Primero Near Leisure Valley Park,4.2,249,Very Good,1042,3905,68,195,1517,1217,"Near Mg Metro Station, Sector 25, Gurgaon",Gurgaon
...,...,...,...,...,...,...,...,...,...,...,...,...
75,Collection O Hotel Sunshine Corporate Suite,5.0,11,Fabulous,729,3059,71,108,1285,857,"Near Medanta Hospital, Gurgaon",Gurgaon
76,SPOT ON Hotel Grace Near Star Mall,3.3,324,Fair,541,2308,72,108,970,646,"Near Bus Stand, Mianwali Colony, Gurgaon",Gurgaon
77,Collection O The Green Acres,4.8,10,Excellent,826,3640,73,108,1447,965,"Near Netaji Subhash Marg, Huda Colony, Gurgaon",Gurgaon
78,Collection O Starlight Sanctuary,4.9,20,Fabulous,906,3768,72,108,1583,1055,"Islampur Colony, Gurgaon",Gurgaon


# Kolkata

In [31]:
titles = []
ratings = []
no_ratings = []
rat = []
actual_prices = [] 
slashed_prices = []
discount = [] 
taxes = []
address = []
links = []
savings_price = []
total_price = []

for i in range(1, 5):
    url = f'https://www.oyorooms.com/hotels-in-kolkata/?page={i}'
    request_header = {'Content-Type': 'text/html; charset=UTF-8','User-Agent': 'Chrome/101.0.0.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/119.0','Accept-Encoding': 'gzip, deflate, br'}
    page = requests.get(url,headers =request_header)
    soup = BeautifulSoup(page.text)

    #titles
    for i in soup.find_all("div", class_ = 'oyo-cell--12-col oyo-cell--8-col-tablet oyo-cell--4-col-phone'):
        title = i.find("h3", class_ = "listingHotelDescription__hotelName d-textEllipsis")
        titles.append(title.text)
        rating_wrapper = i.find("div", class_ = 'hotelRating__wrapper')

        if rating_wrapper: 
            text = rating_wrapper.text
            r1 = re.findall('(\d\.\d)',text)
            if r1:
                ratings.append(r1[0])
            else:
                ratings.append(np.nan)

            r2 = re.findall('\((\d+)',text)
            if r2:
                no_ratings.append(r2[0])
            else:
                no_ratings.append(np.nan)

            r3 = re.findall('\·(\w.+)',text)
            if r3:
                rat.append(r3[0])
            else:
                rat.append(np.nan) 
        else:
            ratings.append(np.nan)
            no_ratings.append(np.nan)
            rat.append(np.nan)

    #prices 
    for i in soup.find_all("div", class_ = 'listingPrice__numbers'):
        ac_price = i.find('span', class_ = 'listingPrice__finalPrice listingPrice__finalPrice--black').text
        sl_price = i.find('span', class_ = 'listingPrice__slashedPrice d-body-lg').text 
        dis = i.find('span', class_ = 'listingPrice__percentage').text
        if ac_price:
            r1 = re.findall('₹(.+)', ac_price)
            if r1:
                actual_prices.append(r1[0])
            else:
                actual_prices.append(np.nan)
        if sl_price:
            r2 = re.findall('₹(.+)',sl_price)
            if r2:
                slashed_prices.append(r2[0])
            else:
                slashed_prices.append(np.nan)
        if dis:
            r3 = re.findall('(.+)%', dis)
            if r3:
                discount.append(r3[0])
            else:
                discount.append(np.nan)

        #taxes        
        for i in soup.find_all("div", class_ = 'listingPrice__perRoomNight'):
            tax = i.text
        if tax:
            r1 = re.findall('₹(\d+)\s?',tax)
            taxes.append(r1[0])
        else:
            taxes.append(np.nan)
            
    #address
    for i in soup.find_all("span", class_ = 'u-line--clamp-2'):
        address.append(i.text)

    #links
    for i in soup.find_all('a',class_="c-nn640c u-width100"):
        link = 'https://www.oyorooms.com' + i.get('href')
        links.append(link)


#prices
for i in links:
    url = i
    request_header = {'Content-Type': 'text/html; charset=UTF-8','User-Agent': 'Chrome/101.0.0.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/119.0','Accept-Encoding': 'gzip, deflate, br'}
    page = requests.get(url,headers =request_header)
    soup = BeautifulSoup(page.text)
    
    for i in soup.find_all('div',class_= 'c-t958pf'):
        text = i.text
        if text:
            r1 = re.findall('Your savings\s?₹(\d+)', text)
            if r1:
                savings_price.append(r1[0]) 
            else:
                savings_price.append(np.nan)

            r2 = re.findall('Total price\s?\s?₹(\d+)', text)
            if r2:
                total_price.append(r2[0]) 
            else:
                total_price.append(np.nan)

In [35]:
dfkolkata = pd.DataFrame({'Title':titles,
'Ratings' :ratings,
'NoofRatings' : no_ratings,
'Overall_Rating': rat,
'Actual_prices':actual_prices,
'Slashed_prices': slashed_prices,
'Discount': discount,
"Tax": taxes,
'Savings_price': savings_price,
'Total_price': total_price,
'Address': address,
'City': ['Kolkata'] * len(titles)}) 
dfkolkata

,Title,Ratings,NoofRatings,Overall_Rating,Actual_prices,Slashed_prices,Discount,Tax,Savings_price,Total_price,Address,City
0,Super Townhouse City Centre 2 Downtown,4.1,169,Very Good,1519,5588,68,170,2171,1741,"Near The Majestic Suites, Newtown, Kolkata",Kolkata
1,Super OYO Flagship Hotel Galaxy,3.8,49,Good,1052,4349,71,170,1827,1218,"Near JIS College, Kolkata",Kolkata
2,Super Collection O New Town DD Block,4.6,150,Excellent,1421,5235,68,170,2034,1631,DD 101 DD block Street no 271 action area 1 N...,Kolkata
3,Super Collection O New Town Near Biswabangla Gate,4.7,88,Excellent,1301,4774,68,170,1855,1487,"Near Biswabangla Gate, Beside Newtown School, ...",Kolkata
4,Collection O Ketan Regency Near Acropolis Mall,4.5,856,Excellent,571,2814,75,170,1034,689,953 Purbachal Road adapt House Prince Park Kal...,Kolkata
...,...,...,...,...,...,...,...,...,...,...,...,...
75,Super OYO Anandalok Guest House,4.2,117,Very Good,1262,4665,68,165,1813,1453,"Champadali, Kolkata",Kolkata
76,Super Townhouse OAK Hotel 1 Lovelock,4.2,1013,Very Good,1722,7067,72,165,2968,1979,"Ballygunge, Kolkata",Kolkata
77,Flagship Hotel Palki Palace Near Netaji Subhas...,3.4,128,Fair,517,2122,71,165,877,609,"Gopalpur I, Kolkata",Kolkata
78,OYO Flagship 79258 Shree Residency,4.1,372,Very Good,709,3014,72,165,1266,844,DD 232 STREET NO 293 ACTION AREA 1 NEW TOWN BI...,Kolkata


# Noida

In [36]:
titles = []
ratings = []
no_ratings = []
rat = []
actual_prices = [] 
slashed_prices = []
discount = [] 
taxes = []
address = []
links = []
savings_price = []
total_price = []

for i in range(1, 5):
    url = f'https://www.oyorooms.com/hotels-in-noida/?page={i}'
    request_header = {'Content-Type': 'text/html; charset=UTF-8','User-Agent': 'Chrome/101.0.0.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/119.0','Accept-Encoding': 'gzip, deflate, br'}
    page = requests.get(url,headers =request_header)
    soup = BeautifulSoup(page.text)

    #titles
    for i in soup.find_all("div", class_ = 'oyo-cell--12-col oyo-cell--8-col-tablet oyo-cell--4-col-phone'):
        title = i.find("h3", class_ = "listingHotelDescription__hotelName d-textEllipsis")
        titles.append(title.text)
        rating_wrapper = i.find("div", class_ = 'hotelRating__wrapper')

        if rating_wrapper: 
            text = rating_wrapper.text
            r1 = re.findall('(\d\.\d)',text)
            if r1:
                ratings.append(r1[0])
            else:
                ratings.append(np.nan)

            r2 = re.findall('\((\d+)',text)
            if r2:
                no_ratings.append(r2[0])
            else:
                no_ratings.append(np.nan)

            r3 = re.findall('\·(\w.+)',text)
            if r3:
                rat.append(r3[0])
            else:
                rat.append(np.nan) 
        else:
            ratings.append(np.nan)
            no_ratings.append(np.nan)
            rat.append(np.nan)

    #prices 
    for i in soup.find_all("div", class_ = 'listingPrice__numbers'):
        ac_price = i.find('span', class_ = 'listingPrice__finalPrice listingPrice__finalPrice--black').text
        sl_price = i.find('span', class_ = 'listingPrice__slashedPrice d-body-lg').text 
        dis = i.find('span', class_ = 'listingPrice__percentage').text
        if ac_price:
            r1 = re.findall('₹(.+)', ac_price)
            if r1:
                actual_prices.append(r1[0])
            else:
                actual_prices.append(np.nan)
        if sl_price:
            r2 = re.findall('₹(.+)',sl_price)
            if r2:
                slashed_prices.append(r2[0])
            else:
                slashed_prices.append(np.nan)
        if dis:
            r3 = re.findall('(.+)%', dis)
            if r3:
                discount.append(r3[0])
            else:
                discount.append(np.nan)

        #taxes        
        for i in soup.find_all("div", class_ = 'listingPrice__perRoomNight'):
            tax = i.text
        if tax:
            r1 = re.findall('₹(\d+)\s?',tax)
            taxes.append(r1[0])
        else:
            taxes.append(np.nan)
            
    #address
    for i in soup.find_all("span", class_ = 'u-line--clamp-2'):
        address.append(i.text)

    #links
    for i in soup.find_all('a',class_="c-nn640c u-width100"):
        link = 'https://www.oyorooms.com' + i.get('href')
        links.append(link)


#prices
for i in links:
    url = i
    request_header = {'Content-Type': 'text/html; charset=UTF-8','User-Agent': 'Chrome/101.0.0.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/119.0','Accept-Encoding': 'gzip, deflate, br'}
    page = requests.get(url,headers =request_header)
    soup = BeautifulSoup(page.text)
    
    for i in soup.find_all('div',class_= 'c-t958pf'):
        text = i.text
        if text:
            r1 = re.findall('Your savings\s?₹(\d+)', text)
            if r1:
                savings_price.append(r1[0]) 
            else:
                savings_price.append(np.nan)

            r2 = re.findall('Total price\s?\s?₹(\d+)', text)
            if r2:
                total_price.append(r2[0]) 
            else:
                total_price.append(np.nan)

In [40]:
dfnoida = pd.DataFrame({'Title':titles,
'Ratings' :ratings,
'NoofRatings' : no_ratings,
'Overall_Rating': rat,
'Actual_prices':actual_prices,
'Slashed_prices': slashed_prices,
'Discount': discount,
"Tax": taxes,
'Savings_price': savings_price, 
'Total_price': total_price,
'Address': address,
'City': ['Noida'] * len(titles)})
dfnoida

,Title,Ratings,NoofRatings,Overall_Rating,Actual_prices,Slashed_prices,Discount,Tax,Savings_price,Total_price,Address,City
0,Collection O The Verve Sector 117,4.4,25,Very Good,782,3546,74,143,1374,916,"Sector 117, Noida",Noida
1,Super Collection O Shivam Hotel,4.6,179,Excellent,867,3247,68,143,1262,1011,"Hazipur, Noida",Noida
2,Townhouse Garhi Chowgandi Near Sector 61 Metro...,4.4,27,Very Good,1033,4238,71,143,1780,1187,"Near Sector 121, Garhi Chaukhandi, Noida",Noida
3,Super Collection O Hotel Noida Stays,4.5,494,Excellent,678,2854,72,143,1199,799,"Opposite Fire Station, Noida",Noida
4,Collection O Hotel Galaxy Inn,4.2,22,Very Good,721,3028,71,143,1272,848,"roza yakubpur, Noida",Noida
...,...,...,...,...,...,...,...,...,...,...,...,...
75,OYO Flagship Hotel Ratiram Palace,3.2,63,Fair,412,2329,78,152,752,501,"street No.5 Near viayak hospital sector 27, N...",Noida
76,Townhouse 1204 Hotel Xanthe,3.5,106,Good,986,4853,76,152,1700,1134,D BLOCK 142 TULSI MARG NOIDA SECTOR 27 OPPOSIT...,Noida
77,Super OYO Hotel Crosswinds Residency,4.5,59,Excellent,676,2559,68,152,995,797,"Near Sapphire International School, Sector 71,...",Noida
78,Townhouse OAK Culture Hotel & restaurant,4.1,86,Very Good,1190,4408,68,152,1713,1373,"Uttar Pradesh , Noida",Noida


# Pune

In [41]:
titles = []
ratings = []
no_ratings = []
rat = []
actual_prices = [] 
slashed_prices = []
discount = [] 
taxes = []
address = []
links = []
savings_price = []
total_price = []

for i in range(1, 5):
    url = f'https://www.oyorooms.com/hotels-in-pune/?page={i}'
    request_header = {'Content-Type': 'text/html; charset=UTF-8','User-Agent': 'Chrome/101.0.0.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/119.0','Accept-Encoding': 'gzip, deflate, br'}
    page = requests.get(url,headers =request_header)
    soup = BeautifulSoup(page.text)

    #titles
    for i in soup.find_all("div", class_ = 'oyo-cell--12-col oyo-cell--8-col-tablet oyo-cell--4-col-phone'):
        title = i.find("h3", class_ = "listingHotelDescription__hotelName d-textEllipsis")
        titles.append(title.text)
        rating_wrapper = i.find("div", class_ = 'hotelRating__wrapper')

        if rating_wrapper: 
            text = rating_wrapper.text
            r1 = re.findall('(\d\.\d)',text)
            if r1:
                ratings.append(r1[0])
            else:
                ratings.append(np.nan)

            r2 = re.findall('\((\d+)',text)
            if r2:
                no_ratings.append(r2[0])
            else:
                no_ratings.append(np.nan)

            r3 = re.findall('\·(\w.+)',text)
            if r3:
                rat.append(r3[0])
            else:
                rat.append(np.nan) 
        else:
            ratings.append(np.nan)
            no_ratings.append(np.nan)
            rat.append(np.nan)

    #prices 
    for i in soup.find_all("div", class_ = 'listingPrice__numbers'):
        ac_price = i.find('span', class_ = 'listingPrice__finalPrice listingPrice__finalPrice--black').text
        sl_price = i.find('span', class_ = 'listingPrice__slashedPrice d-body-lg').text 
        dis = i.find('span', class_ = 'listingPrice__percentage').text
        if ac_price:
            r1 = re.findall('₹(.+)', ac_price)
            if r1:
                actual_prices.append(r1[0])
            else:
                actual_prices.append(np.nan)
        if sl_price:
            r2 = re.findall('₹(.+)',sl_price)
            if r2:
                slashed_prices.append(r2[0])
            else:
                slashed_prices.append(np.nan)
        if dis:
            r3 = re.findall('(.+)%', dis)
            if r3:
                discount.append(r3[0])
            else:
                discount.append(np.nan)

        #taxes        
        for i in soup.find_all("div", class_ = 'listingPrice__perRoomNight'):
            tax = i.text
        if tax:
            r1 = re.findall('₹(\d+)\s?',tax)
            taxes.append(r1[0])
        else:
            taxes.append(np.nan)
            
    #address
    for i in soup.find_all("span", class_ = 'u-line--clamp-2'):
        address.append(i.text)

    #links
    for i in soup.find_all('a',class_="c-nn640c u-width100"):
        link = 'https://www.oyorooms.com' + i.get('href')
        links.append(link)


#prices
for i in links:
    url = i
    request_header = {'Content-Type': 'text/html; charset=UTF-8','User-Agent': 'Chrome/101.0.0.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/119.0','Accept-Encoding': 'gzip, deflate, br'}
    page = requests.get(url,headers =request_header)
    soup = BeautifulSoup(page.text)
    
    for i in soup.find_all('div',class_= 'c-t958pf'):
        text = i.text
        if text:
            r1 = re.findall('Your savings\s?₹(\d+)', text)
            if r1:
                savings_price.append(r1[0]) 
            else:
                savings_price.append(np.nan)

            r2 = re.findall('Total price\s?\s?₹(\d+)', text) 
            if r2:
                total_price.append(r2[0]) 
            else:
                total_price.append(np.nan)

In [44]:
dfpune = pd.DataFrame({'Title':titles,
'Ratings' :ratings,
'NoofRatings' : no_ratings,
'Overall_Rating': rat,
'Actual_prices':actual_prices,
'Slashed_prices': slashed_prices,
'Discount': discount,
"Tax": taxes,
'Savings_price': savings_price,
'Total_price': total_price,
'Address': address,
'City': ['Pune'] * len(titles)})
dfpune

,Title,Ratings,NoofRatings,Overall_Rating,Actual_prices,Slashed_prices,Discount,Tax,Savings_price,Total_price,Address,City
0,Super OYO Sai Avenue,4.5,595,Excellent,672,2790,71,117,988,793,"Pune, Pune",Pune
1,Capital O Hotel Deluxe,3.9,241,Good,538,2669,75,117,979,653,"Pune District, Pune",Pune
2,Super OYO Flagship Hotel Saiinn,4.3,1482,Very Good,578,2452,71,117,1030,687,"Hinjewadi Phase 1, Pune",Pune
3,Flagship Sapphire Homes Stay,4.0,1253,Very Good,589,2498,71,117,1049,700,"Near Laxmi Chowk, Pune",Pune
4,OYO Flagship Spot Light Lodging,3.8,527,Good,499,2137,71,117,898,599,Sr. No. 125 Manjari Mundwa Road Haveli Hadapsa...,Pune
...,...,...,...,...,...,...,...,...,...,...,...,...
75,OYO Flagship Hotel Atithi,4.6,4,Excellent,499,2280,73,80,898,599,"Katraj, Pune",Pune
76,Collection O D'Villa,4.3,50,Very Good,1271,4697,68,80,1825,1463,"Near D mart, Laxman Nagar, Pune",Pune
77,Collection O White Castle,4.0,181,Very Good,1342,5474,71,80,2299,1533,"Maharashtra , Pune",Pune
78,OYO Flagship Kanha Inn Lodging,3.9,64,Good,840,3502,71,80,1471,981,"Next to Dhareshwar Mangal Karyale, Pune",Pune


In [45]:
# concat dfs
dfs = [dfhyd, dfdelhi, dfbnglr, dfmumbai, dfchennai, dfgurgaon, dfnoida, dfpune, dfkolkata]
df = pd.concat(dfs)

In [56]:
df

,Title,Ratings,NoofRatings,Overall_Rating,Actual_prices,Slashed_prices,Discount,Tax,Savings_price,Total_price,Address,City
0,Super Townhouse Oak N7 Elite,4.3,2111,Very Good,1262,4539,68,132,1764,1414,"Champapet, Hyderabad",Hyderabad
1,Super OYO Flagship Vsm Grand,4.6,598,Excellent,864,3237,68,132,1258,1008,"Aprana Sarovar , Gopanpally",Hyderabad
2,Townhouse Secunderabad Near Paradise Metro Sta...,4.5,88,Excellent,1273,5235,72,132,2199,1466,"Jogani, Hyderabad",Hyderabad
3,Super OYO Flagship Boduppal,4.5,2135,Excellent,721,3688,77,132,1257,838,"Near Andhra Bank, Medipally, Hyderabad",Hyderabad
4,OYO Hotel Golden Pride,4.4,1008,Very Good,709,3014,72,132,1266,844,"Near Madhura Nagar, Shamshabad, Hyderabad",Hyderabad
...,...,...,...,...,...,...,...,...,...,...,...,...
75,Super OYO Anandalok Guest House,4.2,117,Very Good,1262,4665,68,165,1813,1453,"Champadali, Kolkata",Kolkata
76,Super Townhouse OAK Hotel 1 Lovelock,4.2,1013,Very Good,1722,7067,72,165,2968,1979,"Ballygunge, Kolkata",Kolkata
77,Flagship Hotel Palki Palace Near Netaji Subhas...,3.4,128,Fair,517,2122,71,165,877,609,"Gopalpur I, Kolkata",Kolkata
78,OYO Flagship 79258 Shree Residency,4.1,372,Very Good,709,3014,72,165,1266,844,DD 232 STREET NO 293 ACTION AREA 1 NEW TOWN BI...,Kolkata


In [47]:
# write df to csv
file_path = 'oyo.csv'
df.to_csv(file_path, index=False)
print("DataFrame has been written to", file_path)

DataFrame has been written to oyo.csv


In [55]:
df.duplicated().sum()

0